In [ ]:
using LinearAlgebra
using Statistics
using IterativeSolvers
using Convex
using SCS
using COSMO
using NLsolve
using DifferentialEquations
using SparseArrays
using Random
using GraphPlot
using MathOptInterface
using DelimitedFiles
using Colors
using Cairo, Compose
using Optim

using JLD2

using LightGraphs
using PyPlot
PyPlot.svg(true);

In [ ]:
function load_cdf(fname)
    # load graph given in IEEE cdf format
    buslines = []
    branchlines = []
    MVA_base = 0.0
    
    open(fname) do file
        state = :nothing
        for (i, line) in enumerate(eachline(file))
            if i == 1
                MVA_base = parse(Float64, line[32:37])
            end
            
            # where in the file are we? set our state accordingly
            if startswith(line, "BUS DATA FOLLOWS")
                state = :bus
                continue
            end
            
            if startswith(line, "BRANCH DATA FOLLOWS")
                state = :branch
                continue
            end   
            
            if startswith(line, "-999")
                state = :nothing
                continue
            end
            
            # separate data 
            if state == :bus
                push!(buslines, line)
            elseif state == :branch
                push!(branchlines, line)
            end
        end
    end
    
    # relevant bus data
    io = IOBuffer(join(buslines, "\n"))
    busdata = readdlm(io)
    close(io)
    
    voltage = busdata[:,8]
    angle = busdata[:,9]
    load_mw = busdata[:,10]
    load_mvar = busdata[:,11]
    gen_mw = busdata[:,12]
    gen_mvar = busdata[:,13]
    base_kv = busdata[:,14]
    
    # relevant branch data
    io = IOBuffer(join(branchlines, "\n"))
    branchdata = readdlm(io, Any)
    close(io)
    
    branch_a = branchdata[:,1]
    branch_b = branchdata[:,2]
    branch_type = branchdata[:,6]
    
    branch_R = branchdata[:,7]
    branch_X = branchdata[:,8]
    
    branch_B = branch_X./(branch_R.^2 + branch_X.^2)
    branch_G = branch_R./(branch_R.^2 + branch_X.^2)
    
    # construct graph
    g = Graph(length(voltage))
    Bs = Float64[]
    Gs = Float64[]
    for (a, b, t, B, G) in zip(branch_a, branch_b, branch_type, branch_B, branch_G)
        if t == 0 # transmission line
            add_edge!(g, a, b)
            push!(Bs, B*abs(voltage[a]*voltage[b]))
            push!(Gs, G*abs(voltage[a]*voltage[b]))
        end
    end
    
    P = gen_mw - load_mw
    P .-= mean(P)
    
    Q = gen_mvar - load_mvar
    
    g, Bs, Gs, P/MVA_base, Q/MVA_base, angle*π/180
end

g, B, G, P, Q, δnet = load_cdf("ieee14cdf.txt")

In [ ]:
sum(Q), sum(P)

In [ ]:
N = nv(g)
E = incidence_matrix(g; oriented=true)
is_connected(g)

In [ ]:
function distance_matrix(g)
    dists = zeros(N, N)

    for i=1:N
        sp = LightGraphs.dijkstra_shortest_paths(g, [i])
        dists[i,:] .= sp.dists
    end
    
    dists
end

dists = distance_matrix(g);

In [ ]:
function optimize_graph(g, B, P, α; δ0=1e-6randn(N), GI=0.0)
    N = nv(g)

    E = incidence_matrix(g; oriented=true)

    # laplacian
    L = Symmetric(E*diagm(0=>B)*E')

    L_sp = dropzeros(sparse(L))

    function f_steady_state(F, δ)
        F .= P - E*(B.*sin.(E'*δ))
    end

    # initial condition
#     δ0 = 1e-6randn(nv(g))
    sln = nlsolve(f_steady_state, δ0; autodiff=:forward)

    # order parameter at the fixed point
    R0 = abs2(mean(exp.(1im*sln.zero)))
    
    @show R0, converged(sln)
    
    # check by solving ODE
    function f_kuramoto(du, u, p, t)
        f_steady_state(du, u)
    end

    # Noisy extension
    δbar = sln.zero
    
    # construct S matrix
    h = complete_graph(nv(g))
    Eh = incidence_matrix(h; oriented=true)
    cosδbar = cos.(Eh'*δbar)
    # cosδbar = ones(ne(h))

    # Laplacian of the corresponding complete graph
    S = Symmetric(-Eh*diagm(0=>cosδbar)*Eh'/nv(g)^2)

    # weighted Laplacian expanded around fixed point
    B_fp = cos.(E'*δbar).*B
    L_fp = Symmetric(E*diagm(0 => B_fp)*E')
    
    # optimize!
    if GI == 0.0
        Eq = Convex.Semidefinite(2N)
        C = Convex.Semidefinite(N)

        Mmat = [zeros(N,N)  I
                -L_fp  -α*I]

        Rmat = [zeros(N,N) zeros(N,N)
                zeros(N,N) -C]

        problem = Convex.maximize(tr(S*Eq[1:N,1:N]), [
    #             L_fp*Eq + Eq*L_fp == C, 
                Mmat*Eq + Eq*Mmat' == Rmat,
                Eq*ones(2N) == zeros(2N),
        #         sum(Eq) == 0.0,
                diag(C, 0) == 1.0,
        #         tr(C) == nv(g),
                ])

        Convex.solve!(problem, () -> COSMO.Optimizer(verbose=false, 
                eps_abs=1e-7, eps_rel=1e-7, max_iter=200000))
    else
        # optimize including integral control
        Eq = Convex.Semidefinite(2N)
        C = Convex.Semidefinite(N)

        Mmat = [zeros(N,N)                     I         
                (-L_fp - GI*ones(N,N)/N)       -α*I]

        Rmat = [zeros(N,N) zeros(N,N)
                zeros(N,N)    -C      ]

        problem = Convex.maximize(tr(S*Eq[1:N,1:N]), [
    #             L_fp*Eq + Eq*L_fp == C, 
                Mmat*Eq + Eq*Mmat' == Rmat,
                C*ones(N) == zeros(N),
#                 Eq[1:2N,1:2N]*ones(2N) == zeros(2N),
        #         sum(Eq) == 0.0,
                diag(C, 0) == 1.0,
        #         tr(C) == nv(g),
                ])

        Convex.solve!(problem, () -> COSMO.Optimizer(verbose=false, 
                eps_abs=1e-7, eps_rel=1e-7, max_iter=200000))        
    end
    
    @show problem.optval, problem.status
    
    if (problem.status == MathOptInterface.OPTIMAL) && converged(sln)
        C.value, Eq.value, S, L_fp, R0, problem.optval, δ0, δbar, f_kuramoto
    else
        [NaN], [NaN], [NaN], [NaN], NaN, NaN, NaN, NaN, nothing
    end
end

In [ ]:
Copt, Eopt, S, L_fp, R0, optval, δ0, δbar, f_kuramoto = optimize_graph(g, B, P, 1.0; GI=2.0, δ0=0.5π*randn(N))

In [ ]:
fig, axs = subplots(1, 3, figsize=(8, 2.5))

ax = axs[1]
cmin, cmax = abs(minimum(Copt)), abs(maximum(Copt))
cm = maximum([cmin, cmax])
sc = ax.matshow(Copt, cmap="RdBu", vmin=-cm, vmax=cm)
fig.colorbar(sc, ax=ax, fraction=0.046, pad=0.04, label="noise covariance matrix C")

ax = axs[2]
cmin, cmax = abs(minimum(Eopt)), abs(maximum(Eopt))
cm = maximum([cmin, cmax])
sc = ax.matshow(Eopt, cmap="RdBu", vmin=-cm, vmax=cm)
fig.colorbar(sc, ax=ax, fraction=0.046, pad=0.04, label="perturbation covariance ⟨εεᵀ⟩")

ax = axs[3]

CD = Copt./dists
CD[.!isfinite.(CD)] .= 0.0
cmin, cmax = abs(minimum(CD)), abs(maximum(CD))
cm = maximum([cmin, cmax])
sc = ax.matshow(CD, cmap="RdBu", vmin=-cm, vmax=cm)
fig.colorbar(sc, ax=ax, fraction=0.046, pad=0.04, label="C/D")

fig.tight_layout()

In [ ]:
@show sum(abs.(Copt))/N^2
@show sum(Eopt)
@show norm(Eopt[1:2N,1:2N]*ones(2N))

# optimal solution
optval

In [ ]:
# compare to uniform noise
C_unif = ((1 + 1/nv(g))I - ones(nv(g), nv(g))/nv(g))

E_uf = lyap(Array(L_fp), -C_unif)

optval_uniform = tr(E_uf*S)

@show optval_uniform
@show optval_uniform/optval

In [ ]:
# jldsave("data/ieee_$(N)_fix_Cii.jld2"; g, K, δ0, δbar, R0, Copt, Eopt, optval, optval_uniform)

In [ ]:
# calculate for various α
αs = 10.0.^LinRange(-2, 2, 101)
Copts = []
Eopts = []
Ss = []
L_fps = []
R0s = []
δ0s = []
δbars = []
optvals = []

δ0 = zeros(nv(g))
for α in αs
    Copt, Eopt, S, L_fp, R0, optval, δ0, δbar, f_kuramoto = optimize_graph(g, B, P, α; δ0=δ0)
    
    push!(Copts, Copt)
    push!(Eopts, Eopt)
    push!(Ss, S)
    push!(L_fps, L_fp)
    push!(R0s, R0)
    push!(optvals, optval)
    push!(δ0s, δ0)
    push!(δbars, δbar)
    
    @show α, optval
end

In [ ]:
# jldsave("data/powergrid_ieee_$(N)_fix_Cii.jld2"; g, αs, Copts, Eopts, Ss, L_fps, R0s, δ0s, δbars, optvals)

In [ ]:
fig, ax = subplots()

Cnorms = [mean(abs.(C)) for C in Copts]

ax.semilogx(αs, optvals)

ax2 = ax.twinx()
ax2.semilogx(αs, Cnorms, "C1")

ax.set_xlabel("α")
ax.set_ylabel("tr(HE)")

In [ ]:
# calculate for various GIs
α = 0.1
GIs = 10.0.^LinRange(-2, 2, 11)
Copts = []
Eopts = []
Ss = []
L_fps = []
R0s = []
δ0s = []
δbars = []
optvals = []

δ0 = zeros(nv(g))
for GI in GIs
    Copt, Eopt, S, L_fp, R0, optval, δ0, δbar, f_kuramoto = optimize_graph(g, B, P, α; δ0=δ0, GI=GI)
    
    push!(Copts, Copt)
    push!(Eopts, Eopt)
    push!(Ss, S)
    push!(L_fps, L_fp)
    push!(R0s, R0)
    push!(optvals, optval)
    push!(δ0s, δ0)
    push!(δbars, δbar)
    
    @show GI, optval
end

In [ ]:
# jldsave("data/powergrid_control_ieee_$(N)_fix_Cii.jld2"; g, α, GIs, Copts, Eopts, Ss, L_fps, R0s, δ0s, δbars, optvals)

In [ ]:
fig, ax = subplots()

Cnorms = [mean(abs.(C)) for C in Copts]

ax.semilogx(GIs, optvals)

ax2 = ax.twinx()
ax2.semilogx(GIs, Cnorms, "C1")

ax.set_xlabel("G_I")
ax.set_ylabel("tr(HE)")

In [ ]:
# find a particular realization of the noise input
function G_from_C(C)
    U, Σ, V = svd(C)
    U*diagm(0 => sqrt.(Σ))*U'
end

G = G_from_C(Copt)
η = G*randn(nv(g));

In [ ]:
# run a SDE solution to check results

function solve_sde(σ, δ0, G; tspan=(0.0, 3000.0), dt=0.01)
    # σ is the noise strength (multiplies the noise matrix)
    function g_kuramoto(du, u, p, t)
        du .= 1/sqrt(2)*σ*G
    end
    
    prob = SDEProblem(f_kuramoto, g_kuramoto, δ0, tspan; noise_rate_prototype=zeros(size(G)...))
    sdesln = solve(prob, EM(), dt=dt)
end

function cumtrapz_avg(t::T, Y::T) where {T <: AbstractVector}
    # Estimates the cumulative time average integral 1/T ∫₀ᵀ f(t) dt using the trapezoid rule
    # where time points are in t and corresponding samples of f are in Y
    
    # Check matching vector length
    @assert length(t) == length(Y)
    
    # Initialize Output
    out = similar(t)
    out[1] = 0.0
    # Iterate over arrays
    for i in 2:length(t)
        out[i] = out[i-1] + 0.5*(t[i] - t[i-1])*(Y[i] + Y[i-1])
    end
    out[2:end] ./= (t[2:end] .- t[1])
    out[1] = out[2]

    out
end

function Rsqr_from_sde(sdesln)
    # numerically integrate and average
    Rsqrs = [abs(mean(exp.(1im*u)))^2 for u in sdesln.u]
end

In [ ]:
σ = 0.25
G = G_from_C(Copt)

sdesln = solve_sde(σ, δbar, G; tspan=(0.0, 5000.0))
Rsqrs_sde = Rsqr_from_sde(sdesln);
Rsqrs_avg = cumtrapz_avg(sdesln.t, Rsqrs_sde);

In [ ]:
G_unif = G_from_C(C_unif)

sdesln_unif = solve_sde(σ, δbar, G_unif; tspan=(0.0, 5000.0))
Rsqrs_sde_unif = Rsqr_from_sde(sdesln_unif);
Rsqrs_avg_unif = cumtrapz_avg(sdesln_unif.t, Rsqrs_sde_unif);

In [ ]:
ts = sdesln.t
ts_unif = sdesln_unif.t

# jldsave("data/ieee_$(N)_timeseries.jld2"; Rsqrs_avg, Rsqrs_avg_unif, ts, ts_unif, σ, G)

In [ ]:
fig, ax = subplots()
# ax.axhline(R0, ls=":", label="R₀²")
# ax.axhline(R0 + 0.5σ^2*optval, label="R₀² + 0.5σ² tr(SE)_opt", color="C1", ls=":")
# ax.axhline(R0 + 0.5σ^2*optval_uniform, label="R₀² + 0.5σ² tr(SE)_unif", color="C2", ls=":")
# ax.axhline(R0 - σ^2*problem.optval)

# ax.set_ylim(R0 - 5σ^2*abs(optval_uniform), R0 + 5σ^2*abs(optval_uniform))

# ax.plot(Rsqrs_sde, alpha=0.5)
ax.plot(sdesln.t, Rsqrs_avg, color="C1", label="⟨R²⟩ optimal")
ax.plot(sdesln_unif.t, Rsqrs_avg_unif, color="C2", label="⟨R²⟩ uniform")

# ax.set_xlim(150000, 200000)

ax.legend()
ax.set_xlabel("time t")
ax.set_ylabel("order parameter")

In [ ]:
cm = colormap("RdBu")[15:85]

# i = 19
i = 6
cols = Int64.(round.(70*(Copt[i,:]/2 .+ 0.5)))
cols[cols .== 0] .= 1

# edgecols = [colorant"white" for i=1:N]
# cols[i] = 35

layout=(args...)->spring_layout(args...; C=0.5, MAXITER=1500)
p = gplot(g, layout=layout, nodefillc=cm[cols], nodelabel=1:N, edgelinewidth=K, EDGELINEWIDTH=3)

In [ ]:
# Compose.draw(SVG("ieee$(N).svg"), p)